# Test: the impact of an AGN power-law continuum
---
What effect does the inclusion of an AGN power-law continuum have upon the recovered SFH?

In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [28]:
import os
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
from itertools import product
import multiprocessing
import pandas as pd

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, compute_mw_age, compute_lw_age, compute_sfr_thresh_age
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

fig_path = "/priv/meggs3/u5708159/ppxftests/figs/"


### Defining the AGN continuum
---
In this work, we approximate the AGN continuum as a simple power law,

$$F_{\rm AGN}(\lambda) = F_0 \lambda^{-\alpha_{\lambda}}$$

where the exponent $\alpha_{\lambda} = 2 -\alpha_{\nu}$ where $\alpha_{\nu}$ can range from 1.2 - 2.0 (Groves et al. 2004). 

To determine reasonable values for $F_0$, we use the tight correlation between the total H$\beta$ luminosity, $L_{\rm H\beta}$, and $L_{\rm NT}$, which is defined as the total luminosity of the AGN continuum between 3000 Å and 9500 Å (Neugebauer et al. 1979, Yee 1980), i.e.,

$$L_{\rm NT} = \int^{9500\,Å}_{3000\,Å} F_{\rm AGN}(\lambda) d\lambda 
= \int^{9500\,Å}_{3000\,Å} F_0 \lambda^{-\alpha_{\lambda}} d\lambda 
= \frac{F_0}{1 - \alpha_\lambda}\left(9500^{1 - \alpha_\lambda} - 3000^{1 - \alpha_\lambda} \right)$$

where $F_{\rm AGN}(\lambda)$ is the AGN continuum. In a sample of AGN including quasars, broad & narrow-line radio galaxies and Seyfert 1 & 2 galaxies, Yee (1980) observed a tight correlation such that 

$$L_{\rm NT} \approx 80 L_{\rm H\beta}$$

*Note: equation from [Peterson textbook. eqn. 5.53, p. 90](https://books.google.com.au/books?id=ok4EFlPMStwC&pg=PA90&lpg=PA90&dq=Lnt+agn+continuum&source=bl&ots=QfVvXob4vM&sig=ACfU3U0x69gKrkN-lALkIu0EROAUh1-1vw&hl=en&sa=X&ved=2ahUKEwjW_8fPvqv1AhWXTWwGHarhALcQ6AF6BAgXEAM#v=onepage&q=Lnt%20agn%20continuum&f=false).* 
Referring to table 2 of Dopita et al. (2015), $\log_{10} L_{\rm H\beta} \sim 39 - 42$ in the NLRs of S7 galaxies.
We therefore use our adopted $L_{\rm H\beta}$ in the emission lines to constrain the strength of the AGN continuum in our mock spectra, so that 

$$ F_0 = \frac{80 L_{\rm H\beta}({1 - \alpha_\lambda})}{\left(9500^{1 - \alpha_\lambda} - 3000^{1 - \alpha_\lambda} \right)} $$



In [7]:
###########################################################################
# Defining the AGN continuum using the method of Yee 1980
###########################################################################
L_NT = 1e41

fig, ax = plt.subplots(nrows=1, ncols=1)
for alpha_nu in [0.3, 0.5, 0.7, 1.1, 1.2, 1.3, 1.5, 1.7, 2.0]:
    # Compute the continuum normalisation
    alpha_lambda = 2 - alpha_nu
    F_lambda_0 = L_NT * (1 - alpha_lambda) / (9500**(1 - alpha_lambda) - 3000**(1 - alpha_lambda))

    # Compute the continuum
    lambda_vals_A = np.linspace(3000, 9500, 1e3)
    F_lambda = F_lambda_0 * lambda_vals_A**(-alpha_lambda)

    ax.plot(lambda_vals_A, F_lambda, label=r"$\alpha_\nu =$" + f"{alpha_nu:.1f}")
    ax.set_xlabel("$\lambda$ (Å)")
    ax.set_ylabel("$F(\lambda)$ (erg/s/Å)")
ax.autoscale(axis="x", tight=True, enable=True)
ax.legend()
ax.axvline(3500, color="grey")
ax.axvline(7000, color="grey")
ax.set_title(r"AGN power-law continuum ($L_{\rm NT} = 10^{41}\,\rm erg\,s^{-1}$)")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'AGN power-law continuum ($L_{\\rm NT} = 10^{41}\\,\\rm erg\\,s^{-1}$)')

# Varying both $L_{\rm NT}$ and $\alpha_\nu$
---

In [35]:
###############################################################################
# Settings
###############################################################################
isochrones = "Padova"
SNR = 100
sigma_star_kms = 250
z = 0.01

# Get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)

# ppxf settings
niters = 100
nthreads = 20

# For analysis
lambda_norm_A = 5000
age_thresh = 1e9  # yr
age_thresh_idx = (np.log10(age_thresh) - np.log10(ages[0])) / (np.log10(ages[1]) - np.log10(ages[0]))
sfr_thresh = 1  # Msun yr^-1


In [36]:
###########################################################################
# Helper function for running MC simulations
###########################################################################
def ppxf_helper(args):
    # Unpack arguments
    seed, spec, spec_err, lambda_vals_A = args
    
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_err)
    spec_noise = spec + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec_noise[0] = -9999
    spec_noise[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_noise, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


In [37]:
###############################################################################
# Load a realistic SFH
###############################################################################
gal = 10
sfh_mw = load_sfh(gal, plotit=True)
M_tot = np.nansum(sfh_mw)

# Create the spectrum here, just so we can check it looks OK
create_mock_spectrum(
    sfh_mass_weighted=sfh_mw,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.23073754e+40, 1.83648883e+40, 1.93822822e+40, ...,
        2.13991140e+40, 2.08699381e+40, 1.80059341e+40]),
 array([1.21932919e+38, 1.84268193e+38, 1.92296605e+38, ...,
        2.12491947e+38, 2.10824974e+38, 1.80795947e+38]),
 array([3500.        , 3500.77462622, 3501.54925243, ..., 6998.98661775,
        6999.76124396, 7000.53587018]))

In [38]:
# Parameters
alpha_nu_vals = np.linspace(0.3, 2.1, 5)  # What is a typical value for low-z Seyfert galaxies?
log_L_NT_vals = np.linspace(42, 44, 5)
# alpha_nu_vals = [0.3, 2.0]
# log_L_NT_vals = [42, 43, 44]

In [39]:
###############################################################################
# Before we run ppxf, plot the spectra
###############################################################################
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 4))
ax.set_xlabel("Observed wavelength (Å)")
ax.set_ylabel(r"$F_\lambda(\lambda)\,(erg\,s^{-1}\,Å^{-1})$")

# Plot spectrum without an AGN continuum added as a "control"
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)
lambda_norm_idx = np.nanargmin(np.abs(lambda_vals_A - lambda_norm_A))

# Add to plot 
ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, label="No AGN continuum", color="k", zorder=10000, linewidth=0.5)

for alpha_nu, log_L_NT in product(alpha_nu_vals, log_L_NT_vals):
    # Create spectrum
    L_NT = 10**log_L_NT
    spec, spec_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw,
        agn_continuum=True, L_NT_erg_s=L_NT, alpha_nu=alpha_nu,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)

    # Add to plot 
    ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, 
                label=r"$\alpha_\nu = %.1f, L_{\rm NT} = 10^{%.1f} \,\rm erg\,s^{-1}$" % (alpha_nu, log_L_NT))

ax.legend(fontsize="x-small")
ax.set_yscale("log")
ax.autoscale(axis="x", tight=True, enable=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
###############################################################################
# Run ppxf without an AGN continuum added as a "control"
###############################################################################
# DataFrame for storing results
df = pd.DataFrame()
log_L_NT = 0
alpha_nu = 0

###########################################################################
# Create spectrum
###########################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)

###########################################################################
# Run ppxf WITHOUT regularisation, using a MC approach
###########################################################################
# Input arguments
seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
with multiprocessing.Pool(nthreads) as pool:
    pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
print(f"Elapsed time in ppxf: {time() - t:.2f} s")

###########################################################################
# Compute mean quantities from the MC runs
###########################################################################
df = pd.DataFrame()

# True quantities
sfh_mw_1D = np.nansum(sfh_mw, axis=0)
sfh_lw = sfh_mw * pp_list[0].stellar_template_norms
sfh_lw_1D = np.nansum(sfh_lw, axis=0)
bin_edges, bin_widths = get_bin_edges_and_widths(isochrones=isochrones)
sfr_1D = sfh_mw_1D / bin_widths
log_age_mw_input, log_age_mw_idx_input = compute_mw_age(sfh_mw, age_thresh, isochrones)
log_age_lw_input, log_age_lw_idx_input = compute_lw_age(sfh_lw, age_thresh, isochrones)
log_sfr_age_input, log_sfr_age_idx_input = compute_sfr_thresh_age(sfh_mw, sfr_thresh, isochrones)

# Lists containing the SFHs and mean SFRs from each MC iteration
sfh_MC_mw_list = [pp.weights_mass_weighted for pp in pp_list]
sfh_MC_mw_1D_list = [pp.sfh_mw_1D for pp in pp_list]
sfh_MC_lw_list = [pp.weights_light_weighted for pp in pp_list]
sfh_MC_lw_1D_list = [pp.sfh_lw_1D for pp in pp_list]
sfr_MC_mw_list = [pp.sfr_mean for pp in pp_list]

# Compute the mean SFH and SFR from the lists of MC runs
sfh_MC_mw_mean = np.nansum(np.array(sfh_MC_mw_list), axis=0) / len(sfh_MC_mw_list)
sfh_MC_mw_1D_mean = np.nansum(sfh_MC_mw_mean, axis=0)
sfh_MC_lw_mean = np.nansum(np.array(sfh_MC_lw_list), axis=0) / len(sfh_MC_lw_list)
sfh_MC_lw_1D_mean = np.nansum(sfh_MC_lw_mean, axis=0)
sfr_MC_mw_mean = np.nansum(np.array(sfr_MC_mw_list), axis=0) / len(sfr_MC_mw_list)

# Compute the mean mass- and light-weighted ages below 1 Gyr
log_age_mw_idx_MC_list = [compute_mw_age(sfh, age_thresh, isochrones)[1] for sfh in sfh_MC_mw_list]
log_age_mw_idx_MC_mean = np.nanmean(log_age_mw_idx_MC_list)
log_age_mw_idx_MC_std = np.nanstd(log_age_mw_idx_MC_list)
log_age_lw_idx_MC_list = [compute_lw_age(sfh, age_thresh, isochrones)[1] for sfh in sfh_MC_lw_list]
log_age_lw_idx_MC_mean = np.nanmean(log_age_lw_idx_MC_list)
log_age_lw_idx_MC_std = np.nanstd(log_age_lw_idx_MC_list)
log_age_mw_MC_list = [compute_mw_age(sfh, age_thresh, isochrones)[0] for sfh in sfh_MC_mw_list]
log_age_mw_MC_mean = np.nanmean(log_age_mw_MC_list)
log_age_mw_MC_std = np.nanstd(log_age_mw_MC_list)
log_age_lw_MC_list = [compute_lw_age(sfh, age_thresh, isochrones)[0] for sfh in sfh_MC_lw_list]
log_age_lw_MC_mean = np.nanmean(log_age_lw_MC_list)
log_age_lw_MC_std = np.nanstd(log_age_lw_MC_list)

# Compute the "SFR age"
log_sfr_age_idx_MC_list = [compute_sfr_thresh_age(sfh, sfr_thresh, isochrones)[1] for sfh in sfh_MC_mw_list]
log_sfr_age_idx_MC_median = np.nanmedian(log_sfr_age_idx_MC_list)
log_sfr_age_idx_MC_mean = np.nanmean(log_sfr_age_idx_MC_list)
log_sfr_age_idx_MC_std = np.nanstd(log_sfr_age_idx_MC_list)
log_sfr_age_MC_list = [compute_sfr_thresh_age(sfh, sfr_thresh, isochrones)[0] for sfh in sfh_MC_mw_list]
log_sfr_age_MC_median = np.nanmedian(log_sfr_age_MC_list)
log_sfr_age_MC_mean = np.nanmean(log_sfr_age_MC_list)
log_sfr_age_MC_std = np.nanstd(log_sfr_age_MC_list)

###########################################################################
# Add variables to DataFrame
###########################################################################
df = df.append({
    "AGN continuum?": False,
    "log L_NT": log_L_NT,
    "alpha_nu": alpha_nu,
    
    f"log mass-weighted mean age < {age_thresh:.2e} yr (input)": log_age_mw_input,
    f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) mean": log_age_mw_MC_mean,
    f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) std": log_age_mw_MC_std,
    
    f"log light-weighted mean age < {age_thresh:.2e} yr (input)": log_age_lw_input,
    f"log light-weighted mean age < {age_thresh:.2e} yr (MC) mean": log_age_lw_MC_mean,
    f"log light-weighted mean age < {age_thresh:.2e} yr (MC) std": log_age_lw_MC_std,
    
    f"log SFR age ({sfr_thresh:.1f} Msun yr^-1) (input)": log_sfr_age_input,
    f"log SFR age ({sfr_thresh:.1f} Msun yr^-1) (MC) mean": log_sfr_age_MC_mean,
    f"log SFR age ({sfr_thresh:.1f} Msun yr^-1) (MC) median": log_sfr_age_MC_median,
    f"log SFR age ({sfr_thresh:.1f} Msun yr^-1) (MC) std": log_sfr_age_MC_std,
    
}, ignore_index=True)

###########################################################################
# Plot the input mass- and light-weighted SFHs
###########################################################################
plt.close("all")
pp = PdfPages(os.path.join(fig_path, "agn_continuum", f"gal{gal:004}_SNR={SNR:d}_z={z:.2f}_sigma_star={sigma_star_kms:d}.pdf"))

fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(12, 25))
fig.subplots_adjust(hspace=0.35)

plot_sfh_mass_weighted(sfh_mw, ages, metallicities, ax=axs[0])
axs[0].set_title(f"Galaxy {gal:004} " + r"- $M_{\rm tot} = %.4e\,\rm M_\odot$" % M_tot)

plot_sfh_light_weighted(sfh_lw, ages, metallicities, ax=axs[1])
axs[1].set_title(f"Galaxy {gal:004} " + r"- $M_{\rm tot} = %.4e\,\rm M_\odot$" % M_tot)

###########################################################################
# Plot the mass-weighted weights, summed over the metallicity dimension
###########################################################################
log_scale = True
axs[2].set_title(f"Mass-weighted template weights (S/N = {SNR})")

# Plot the SFHs from each ppxf run, plus the "truth" SFH
axs[2].fill_between(range(N_ages), sfh_mw_1D, step="mid", alpha=0.5, color="lightblue", label="Input SFH")
for jj in range(niters):
    axs[2].step(range(N_ages), sfh_MC_mw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
axs[2].step(range(N_ages), sfh_MC_mw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
# axs[2].step(range(N_ages), sfh_fit_mw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")
axs[2].set_ylim([1e3, None])

# Plot horizontal error bars indicating the SFR threshold age from the MC simulations
y1, y2 = axs[2].get_ylim()
y = 10**(0.9 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.9 * y2
axs[2].errorbar(x=log_sfr_age_idx_MC_median, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
            marker="^", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
            label="SFR age (median, MC simulations)")
axs[2].errorbar(x=log_sfr_age_idx_MC_mean, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
            marker="^", mfc="orange", mec="orange", ecolor="orange", linestyle="none", capsize=10,
            label="SFR age (mean, MC simulations)")
axs[2].errorbar(x=log_sfr_age_idx_input, y=y, xerr=0, yerr=0, 
            marker="^", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
            label="SFR age (input)")

# Plot horizontal error bars indicating the mean mass-weighted age from the MC simulations
y = 10**(0.8 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.8 * y2
axs[2].errorbar(x=log_age_mw_idx_MC_mean, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
            marker="D", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
            label="Mean MW age < 1 Gyr (MC simulations)")
axs[2].errorbar(x=log_age_mw_idx_input, y=y, xerr=0, yerr=0, 
            marker="D", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
            label="Mean MW age < 1 Gyr (input)")

# Plot horizontal error bars indicating the mean light-weighted age from the MC simulations
y = 10**(0.7 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.7 * y2
axs[2].errorbar(x=log_age_lw_idx_MC_mean, y=y, xerr=log_age_lw_idx_MC_std, yerr=0, 
            marker="X", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
            label="Mean LW age < 1 Gyr (MC simulations)")
axs[2].errorbar(x=log_age_lw_idx_input, y=y, xerr=0, yerr=0, 
            marker="X", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
            label="Mean LW age < 1 Gyr (input)")

axs[2].axvline(age_thresh_idx, color="black", linestyle="--", label="Age threshold")

# Decorations 
axs[2].set_xticks(range(N_ages))
axs[2].set_xlabel("Age (Myr)")
axs[2].set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
axs[2].autoscale(axis="x", enable=True, tight=True)
axs[2].set_ylabel(r"Template weight ($\rm M_\odot$)")
axs[2].legend(fontsize="small", loc="center left", bbox_to_anchor=(1.01, 0.5))
axs[2].set_xlabel("Age (Myr)")
axs[2].set_yscale("log") if log_scale else None
axs[2].grid()

###########################################################################
# Plot the light-weighted weights, summed over the metallicity dimension
###########################################################################
log_scale = True
axs[3].set_title(f"Light-weighted template weights (S/N = {SNR})")

# Plot the SFHs from each ppxf run, plus the "truth" SFH
axs[3].fill_between(range(N_ages), sfh_lw_1D, step="mid", alpha=0.5, color="lightblue", label="Input SFH")
for jj in range(niters):
    axs[3].step(range(N_ages), sfh_MC_lw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
axs[3].step(range(N_ages), sfh_MC_lw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
#     axs[3].step(range(N_ages), sfh_fit_lw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")
axs[3].set_ylim([1e35, None])

# Plot horizontal error bars indicating the SFR threshold age from the MC simulations
y1, y2 = axs[3].get_ylim()
y = 10**(0.9 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.9 * y2
axs[3].errorbar(x=log_sfr_age_idx_MC_median, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
            marker="^", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
            label="SFR age (median, MC simulations)")
axs[3].errorbar(x=log_sfr_age_idx_MC_mean, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
            marker="^", mfc="orange", mec="orange", ecolor="orange", linestyle="none", capsize=10,
            label="SFR age (mean, MC simulations)")
axs[3].errorbar(x=log_sfr_age_idx_input, y=y, xerr=0, yerr=0, 
            marker="^", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
            label="SFR age (input)")

# Plot horizontal error bars indicating the mean mass-weighted age from the MC simulations
y = 10**(0.8 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.8 * y2
axs[3].errorbar(x=log_age_mw_idx_MC_mean, y=y, xerr=log_age_mw_idx_MC_std, yerr=0, 
            marker="D", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
            label="Mean MW age < 1 Gyr (MC simulations)")
axs[3].errorbar(x=log_age_mw_idx_input, y=y, xerr=0, yerr=0, 
            marker="D", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
            label="Mean MW age < 1 Gyr (input)")

# Plot horizontal error bars indicating the mean light-weighted age from the MC simulations
y = 10**(0.7 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.7 * y2
axs[3].errorbar(x=log_age_lw_idx_MC_mean, y=y, xerr=log_age_lw_idx_MC_std, yerr=0, 
            marker="X", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
            label="Mean LW age < 1 Gyr (MC simulations)")
axs[3].errorbar(x=log_age_lw_idx_input, y=y, xerr=0, yerr=0, 
            marker="X", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
            label="Mean LW age < 1 Gyr (input)")

axs[3].axvline(age_thresh_idx, color="black", linestyle="--", label="Age threshold")

# Decorations 
axs[3].set_xticks(range(N_ages))
axs[3].set_xlabel("Age (Myr)")
axs[3].set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
axs[3].autoscale(axis="x", enable=True, tight=True)
axs[3].set_ylabel(r"Template weight ($\rm M_\odot\,erg\,s^{-1}\,Å^{-1}$)")
axs[3].legend(fontsize="small", loc="center left", bbox_to_anchor=(1.01, 0.5))
axs[3].set_xlabel("Age (Myr)")
axs[3].set_yscale("log") if log_scale else None
axs[3].grid()

###########################################################################
# Plot the average SFR in each bin
###########################################################################
log_scale = True
axs[4].set_title(f"Mean SFR (S/N = {SNR})")

# Plot the SFHs from each ppxf run, plus the "truth" SFH
axs[4].fill_between(range(N_ages), sfr_1D, step="mid", alpha=0.5, color="lightblue", label="Input mean SFR")
for jj in range(niters):
    axs[4].step(range(N_ages), sfr_MC_mw_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
axs[4].step(range(N_ages), sfr_MC_mw_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
axs[4].set_ylim([1e-2, None])

# Plot horizontal error bars indicating the SFR threshold age from the MC simulations
y1, y2 = axs[4].get_ylim()
y = 10**(0.9 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.9 * y2
# Tracer()()
axs[4].errorbar(x=log_sfr_age_idx_MC_median, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
            marker="^", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
            label="SFR age (median, MC simulations)")
axs[4].errorbar(x=log_sfr_age_idx_MC_mean, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
            marker="^", mfc="orange", mec="orange", ecolor="orange", linestyle="none", capsize=10,
            label="SFR age (mean, MC simulations)")
axs[4].errorbar(x=log_sfr_age_idx_input, y=y, xerr=0, yerr=0, 
            marker="D", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
            label="SFR age (input)")
axs[4].axhline(sfr_thresh, color="black", linestyle=":", label="SFR threshold")

# Decorations 
axs[4].set_xticks(range(N_ages))
axs[4].set_xlabel("Age (Myr)")
axs[4].set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
axs[4].autoscale(axis="x", enable=True, tight=True)
axs[4].set_ylabel(r"Mean SFR ($\rm M_\odot \, yr^{-1}$)")
axs[4].legend(fontsize="small", loc="center left", bbox_to_anchor=(1.01, 0.5))
axs[4].set_xlabel("Age (Myr)")
axs[4].set_yscale("log") if log_scale else None
axs[4].grid()

pp.savefig(fig, bbox_inches="tight")

Running ppxf on 20 threads...



Elapsed time in ppxf: 1082.47 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
###############################################################################
# The effect of the strength and exponent of the AGN continuum on the recovered SFH
###############################################################################
for aa, alpha_nu in enumerate(alpha_nu_vals):
    for ll, log_L_NT in enumerate(log_L_NT_vals):
        ###########################################################################
        # Create spectrum
        ###########################################################################
        spec, spec_err, lambda_vals_A = create_mock_spectrum(
            sfh_mass_weighted=sfh_mw,
            agn_continuum=True, L_NT_erg_s=10**log_L_NT, alpha_nu=alpha_nu,
            isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
            plotit=False)
        
        ###########################################################################
        # Run ppxf WITHOUT regularisation, using a MC approach
        ###########################################################################
        # Input arguments
        seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
        args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

        # Run in parallel
        print(f"Running ppxf on {nthreads} threads...")
        t = time()
        with multiprocessing.Pool(nthreads) as pool:
            pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
        print(f"Elapsed time in ppxf: {time() - t:.2f} s")

        ###########################################################################
        # Compute derived quantities from the ppxf runs
        ###########################################################################        
        # Lists containing the SFHs and mean SFRs from each MC iteration
        sfh_MC_mw_list = [pp.weights_mass_weighted for pp in pp_list]
        sfh_MC_mw_1D_list = [pp.sfh_mw_1D for pp in pp_list]
        sfh_MC_lw_list = [pp.weights_light_weighted for pp in pp_list]
        sfh_MC_lw_1D_list = [pp.sfh_lw_1D for pp in pp_list]
        sfr_MC_mw_list = [pp.sfr_mean for pp in pp_list]

        # Compute the mean SFH and SFR from the lists of MC runs
        sfh_MC_mw_mean = np.nansum(np.array(sfh_MC_mw_list), axis=0) / len(sfh_MC_mw_list)
        sfh_MC_mw_1D_mean = np.nansum(sfh_MC_mw_mean, axis=0)
        sfh_MC_lw_mean = np.nansum(np.array(sfh_MC_lw_list), axis=0) / len(sfh_MC_lw_list)
        sfh_MC_lw_1D_mean = np.nansum(sfh_MC_lw_mean, axis=0)
        sfr_MC_mw_mean = np.nansum(np.array(sfr_MC_mw_list), axis=0) / len(sfr_MC_mw_list)

        # Compute the mean mass- and light-weighted ages below 1 Gyr
        log_age_mw_idx_MC_list = [compute_mw_age(sfh, age_thresh, isochrones)[1] for sfh in sfh_MC_mw_list]
        log_age_mw_idx_MC_mean = np.nanmean(log_age_mw_idx_MC_list)
        log_age_mw_idx_MC_std = np.nanstd(log_age_mw_idx_MC_list)
        log_age_lw_idx_MC_list = [compute_lw_age(sfh, age_thresh, isochrones)[1] for sfh in sfh_MC_lw_list]
        log_age_lw_idx_MC_mean = np.nanmean(log_age_lw_idx_MC_list)
        log_age_lw_idx_MC_std = np.nanstd(log_age_lw_idx_MC_list)
        log_age_mw_MC_list = [compute_mw_age(sfh, age_thresh, isochrones)[0] for sfh in sfh_MC_mw_list]
        log_age_mw_MC_mean = np.nanmean(log_age_mw_MC_list)
        log_age_mw_MC_std = np.nanstd(log_age_mw_MC_list)
        log_age_lw_MC_list = [compute_lw_age(sfh, age_thresh, isochrones)[0] for sfh in sfh_MC_lw_list]
        log_age_lw_MC_mean = np.nanmean(log_age_lw_MC_list)
        log_age_lw_MC_std = np.nanstd(log_age_lw_MC_list)

        # Compute the "SFR age"
        log_sfr_age_idx_MC_list = [compute_sfr_thresh_age(sfh, sfr_thresh, isochrones)[1] for sfh in sfh_MC_mw_list]
        log_sfr_age_idx_MC_median = np.nanmedian(log_sfr_age_idx_MC_list)
        log_sfr_age_idx_MC_mean = np.nanmean(log_sfr_age_idx_MC_list)
        log_sfr_age_idx_MC_std = np.nanstd(log_sfr_age_idx_MC_list)
        log_sfr_age_MC_list = [compute_sfr_thresh_age(sfh, sfr_thresh, isochrones)[0] for sfh in sfh_MC_mw_list]
        log_sfr_age_MC_median = np.nanmedian(log_sfr_age_MC_list)
        log_sfr_age_MC_mean = np.nanmean(log_sfr_age_MC_list)
        log_sfr_age_MC_std = np.nanstd(log_sfr_age_MC_list)

        ###########################################################################
        # Add variables to DataFrame
        ###########################################################################
        df = df.append({
            "AGN continuum?": False,
            "log L_NT": log_L_NT,
            "alpha_nu": alpha_nu,

            f"log mass-weighted mean age < {age_thresh:.2e} yr (input)": log_age_mw_input,
            f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) mean": log_age_mw_MC_mean,
            f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) std": log_age_mw_MC_std,

            f"log light-weighted mean age < {age_thresh:.2e} yr (input)": log_age_lw_input,
            f"log light-weighted mean age < {age_thresh:.2e} yr (MC) mean": log_age_lw_MC_mean,
            f"log light-weighted mean age < {age_thresh:.2e} yr (MC) std": log_age_lw_MC_std,

            f"log SFR age ({sfr_thresh:.1f} Msun yr^-1) (input)": log_sfr_age_input,
            f"log SFR age ({sfr_thresh:.1f} Msun yr^-1) (MC) mean": log_sfr_age_MC_mean,
            f"log SFR age ({sfr_thresh:.1f} Msun yr^-1) (MC) median": log_sfr_age_MC_median,
            f"log SFR age ({sfr_thresh:.1f} Msun yr^-1) (MC) std": log_sfr_age_MC_std,

        }, ignore_index=True)

        ###########################################################################
        # Plot the input mass- and light-weighted SFHs
        ###########################################################################
        fig, axs = plt.subplots(nrows=5, ncols=1, figsize=(12, 25))
        fig.subplots_adjust(hspace=0.35)
        fig.suptitle(r"AGN continuum: $\alpha_\nu = %.1f, \, \log_{10}L_{\rm NT} = %.2f$" % (alpha_nu, log_L_NT))

        plot_sfh_mass_weighted(sfh_mw, ages, metallicities, ax=axs[0])
        axs[0].set_title(f"Galaxy {gal:004} " + r"- $M_{\rm tot} = %.4e\,\rm M_\odot$" % M_tot)

        plot_sfh_light_weighted(sfh_lw, ages, metallicities, ax=axs[1])
        axs[1].set_title(f"Galaxy {gal:004} " + r"- $M_{\rm tot} = %.4e\,\rm M_\odot$" % M_tot)

        ###########################################################################
        # Plot the mass-weighted weights, summed over the metallicity dimension
        ###########################################################################
        log_scale = True
        axs[2].set_title(f"Mass-weighted template weights (S/N = {SNR})")

        # Plot the SFHs from each ppxf run, plus the "truth" SFH
        axs[2].fill_between(range(N_ages), sfh_mw_1D, step="mid", alpha=0.5, color="lightblue", label="Input SFH")
        for jj in range(niters):
            axs[2].step(range(N_ages), sfh_MC_mw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
        axs[2].step(range(N_ages), sfh_MC_mw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
        # axs[2].step(range(N_ages), sfh_fit_mw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")
        axs[2].set_ylim([1e3, None])

        # Plot horizontal error bars indicating the SFR threshold age from the MC simulations
        y1, y2 = axs[2].get_ylim()
        y = 10**(0.9 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.9 * y2
        axs[2].errorbar(x=log_sfr_age_idx_MC_median, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
                    marker="^", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
                    label="SFR age (median, MC simulations)")
        axs[2].errorbar(x=log_sfr_age_idx_MC_mean, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
                    marker="^", mfc="orange", mec="orange", ecolor="orange", linestyle="none", capsize=10,
                    label="SFR age (mean, MC simulations)")
        axs[2].errorbar(x=log_sfr_age_idx_input, y=y, xerr=0, yerr=0, 
                    marker="^", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
                    label="SFR age (input)")

        # Plot horizontal error bars indicating the mean mass-weighted age from the MC simulations
        y = 10**(0.8 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.8 * y2
        axs[2].errorbar(x=log_age_mw_idx_MC_mean, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
                    marker="D", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
                    label="Mean MW age < 1 Gyr (MC simulations)")
        axs[2].errorbar(x=log_age_mw_idx_input, y=y, xerr=0, yerr=0, 
                    marker="D", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
                    label="Mean MW age < 1 Gyr (input)")

        # Plot horizontal error bars indicating the mean light-weighted age from the MC simulations
        y = 10**(0.7 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.7 * y2
        axs[2].errorbar(x=log_age_lw_idx_MC_mean, y=y, xerr=log_age_lw_idx_MC_std, yerr=0, 
                    marker="X", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
                    label="Mean LW age < 1 Gyr (MC simulations)")
        axs[2].errorbar(x=log_age_lw_idx_input, y=y, xerr=0, yerr=0, 
                    marker="X", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
                    label="Mean LW age < 1 Gyr (input)")

        axs[2].axvline(age_thresh_idx, color="black", linestyle="--", label="Age threshold")

        # Decorations 
        axs[2].set_xticks(range(N_ages))
        axs[2].set_xlabel("Age (Myr)")
        axs[2].set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
        axs[2].autoscale(axis="x", enable=True, tight=True)
        axs[2].set_ylabel(r"Template weight ($\rm M_\odot$)")
        axs[2].legend(fontsize="small", loc="center left", bbox_to_anchor=(1.01, 0.5))
        axs[2].set_xlabel("Age (Myr)")
        axs[2].set_yscale("log") if log_scale else None
        axs[2].grid()

        ###########################################################################
        # Plot the light-weighted weights, summed over the metallicity dimension
        ###########################################################################
        log_scale = True
        axs[3].set_title(f"Light-weighted template weights (S/N = {SNR})")

        # Plot the SFHs from each ppxf run, plus the "truth" SFH
        axs[3].fill_between(range(N_ages), sfh_lw_1D, step="mid", alpha=0.5, color="lightblue", label="Input SFH")
        for jj in range(niters):
            axs[3].step(range(N_ages), sfh_MC_lw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
        axs[3].step(range(N_ages), sfh_MC_lw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
        #     axs[3].step(range(N_ages), sfh_fit_lw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")
        axs[3].set_ylim([1e35, None])

        # Plot horizontal error bars indicating the SFR threshold age from the MC simulations
        y1, y2 = axs[3].get_ylim()
        y = 10**(0.9 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.9 * y2
        axs[3].errorbar(x=log_sfr_age_idx_MC_median, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
                    marker="^", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
                    label="SFR age (median, MC simulations)")
        axs[3].errorbar(x=log_sfr_age_idx_MC_mean, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
                    marker="^", mfc="orange", mec="orange", ecolor="orange", linestyle="none", capsize=10,
                    label="SFR age (mean, MC simulations)")
        axs[3].errorbar(x=log_sfr_age_idx_input, y=y, xerr=0, yerr=0, 
                    marker="^", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
                    label="SFR age (input)")

        # Plot horizontal error bars indicating the mean mass-weighted age from the MC simulations
        y = 10**(0.8 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.8 * y2
        axs[3].errorbar(x=log_age_mw_idx_MC_mean, y=y, xerr=log_age_mw_idx_MC_std, yerr=0, 
                    marker="D", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
                    label="Mean MW age < 1 Gyr (MC simulations)")
        axs[3].errorbar(x=log_age_mw_idx_input, y=y, xerr=0, yerr=0, 
                    marker="D", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
                    label="Mean MW age < 1 Gyr (input)")

        # Plot horizontal error bars indicating the mean light-weighted age from the MC simulations
        y = 10**(0.7 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.7 * y2
        axs[3].errorbar(x=log_age_lw_idx_MC_mean, y=y, xerr=log_age_lw_idx_MC_std, yerr=0, 
                    marker="X", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
                    label="Mean LW age < 1 Gyr (MC simulations)")
        axs[3].errorbar(x=log_age_lw_idx_input, y=y, xerr=0, yerr=0, 
                    marker="X", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
                    label="Mean LW age < 1 Gyr (input)")

        axs[3].axvline(age_thresh_idx, color="black", linestyle="--", label="Age threshold")

        # Decorations 
        axs[3].set_xticks(range(N_ages))
        axs[3].set_xlabel("Age (Myr)")
        axs[3].set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
        axs[3].autoscale(axis="x", enable=True, tight=True)
        axs[3].set_ylabel(r"Template weight ($\rm M_\odot\,erg\,s^{-1}\,Å^{-1}$)")
        axs[3].legend(fontsize="small", loc="center left", bbox_to_anchor=(1.01, 0.5))
        axs[3].set_xlabel("Age (Myr)")
        axs[3].set_yscale("log") if log_scale else None
        axs[3].grid()

        ###########################################################################
        # Plot the average SFR in each bin
        ###########################################################################
        log_scale = True
        axs[4].set_title(f"Mean SFR (S/N = {SNR})")

        # Plot the SFHs from each ppxf run, plus the "truth" SFH
        axs[4].fill_between(range(N_ages), sfr_1D, step="mid", alpha=0.5, color="lightblue", label="Input mean SFR")
        for jj in range(niters):
            axs[4].step(range(N_ages), sfr_MC_mw_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
        axs[4].step(range(N_ages), sfr_MC_mw_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
        axs[4].set_ylim([1e-2, None])

        # Plot horizontal error bars indicating the SFR threshold age from the MC simulations
        y1, y2 = axs[4].get_ylim()
        y = 10**(0.9 * (np.log10(y2) - np.log10(y1)) + np.log10(y1)) if log_scale else 0.9 * y2
        # Tracer()()
        axs[4].errorbar(x=log_sfr_age_idx_MC_median, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
                    marker="^", mfc="red", mec="red", ecolor="red", linestyle="none", capsize=10,
                    label="SFR age (median, MC simulations)")
        axs[4].errorbar(x=log_sfr_age_idx_MC_mean, y=y, xerr=log_sfr_age_idx_MC_std, yerr=0, 
                    marker="^", mfc="orange", mec="orange", ecolor="orange", linestyle="none", capsize=10,
                    label="SFR age (mean, MC simulations)")
        axs[4].errorbar(x=log_sfr_age_idx_input, y=y, xerr=0, yerr=0, 
                    marker="D", mfc="lightblue", mec="lightblue", ecolor="lightblue", linestyle="none",
                    label="SFR age (input)")
        axs[4].axhline(sfr_thresh, color="black", linestyle=":", label="SFR threshold")

        # Decorations 
        axs[4].set_xticks(range(N_ages))
        axs[4].set_xlabel("Age (Myr)")
        axs[4].set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
        axs[4].autoscale(axis="x", enable=True, tight=True)
        axs[4].set_ylabel(r"Mean SFR ($\rm M_\odot \, yr^{-1}$)")
        axs[4].legend(fontsize="small", loc="center left", bbox_to_anchor=(1.01, 0.5))
        axs[4].set_xlabel("Age (Myr)")
        axs[4].set_yscale("log") if log_scale else None
        axs[4].grid()
        
        # Write to pdf
        pp.savefig(fig, bbox_inches="tight")
        plt.close("all")
        
pp.close()
print("Finished!")     

Running ppxf on 20 threads...



Elapsed time in ppxf: 1079.31 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...



Elapsed time in ppxf: 1075.90 s


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...



Elapsed time in ppxf: 1430.08 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Finished!


In [50]:
########################################################
# Plot the recovered mass-weighted mean age vs. alpha_nu
########################################################
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.15, 0.5, 0.75])

# Actual value
ax.axhline(df[f"log mass-weighted mean age < {age_thresh:.2e} yr (input)"].unique()[0], color="gray", label="True value")

# MC run w/o AGN continuum
cond = df["AGN continuum?"] == False
ax.axhspan(ymin=df.loc[cond, f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) mean"].values[0] - df.loc[cond, f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) std"].values[0],
           ymax=df.loc[cond, f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) mean"].values[0] + df.loc[cond, f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) std"].values[0],
           color="pink", alpha=0.5)
ax.axhline(df.loc[cond, f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) mean"].values[0], color="pink", label="no AGN continuum")

# Results from MC runs
for alpha_nu in alpha_nu_vals:
    cond = df["alpha_nu"] == alpha_nu
    ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
                y=df.loc[cond, f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) mean"].values, 
                yerr=df.loc[cond, f"log mass-weighted mean age < {age_thresh:.2e} yr (MC) std"].values,
                linestyle="none", marker="D", label=r"$\alpha_\nu = %.1f$" % alpha_nu, zorder=999)

# Decorations
ax.set_xlabel(r"$\log_{10} L_{\rm NT}$")
ax.set_ylabel(r"Mass-weighted mean age < 1 Gyr (log yr)")
ax.grid()
ax.legend(bbox_to_anchor=[1.05, 0.5], loc="center left")
ax.set_title(r"Effect of AGN continuum")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Effect of AGN continuum')